In [1]:
import os
import random

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import SimpleITK as sitk
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import monai.transforms as monai_transforms

from dataset import MRIDataset, get_loader
from models import C3D, generate_model, ResNet
from train import epoch_iter, add_metrics, save_checkpoint, get_metrics

# Preprocessing

In [2]:
"/data1/TBM/data_for_AI/data/spatial_normalized_pt_1/"
basepath = "/data1/TBM/data_for_AI/data/realigned_pt/"
csvpath = '/data1/TBM/data_for_AI/subjects_info/final_TBM_subjects_info.csv'

form = 'process'

# use for raw file
modality = "T2s"
use_file = "R2S.nii"

# use for process file
prefix = 'r'
suffix = '_fdt_paths.nii'

random_state = 65489132
torch.manual_seed(random_state)
random.seed(random_state)
np.random.seed(random_state)

In [3]:
df_data = pd.read_csv(csvpath)

filenames = []
labels_data = []

mean_data = []
var_data = []

def get_path_and_label(name, label, form = 'raw', prefix = 'wr', suffix = '_fdt_paths.nii'):
    
    if form == 'raw':
        if label.lower() == 'mci':
            category = 1
            fdt_paths_path = os.path.join(basepath, 'MCI',name, modality, use_file)
        elif label.lower() == 'normal':
            category = 0
            fdt_paths_path = os.path.join(basepath, 'Normal',name, modality, use_file)
        elif label.lower() == 'mmd':
            category = 2
            fdt_paths_path = os.path.join(basepath, 'AD',name, modality, use_file)
        else:
            raise ValueError(f"No label name {label}")

    else:
        if label.lower() == 'mci':
            category = 1
        elif label.lower() == 'normal':
            category = 0

        elif label.lower() == 'mmd':
            category = 2
        else:
            raise ValueError(f"No label name {label}")
        fdt_paths_path = os.path.join(basepath, prefix + name + suffix)
    return fdt_paths_path, category


first_shape_flag = False
first_shape = (0,0,0)

for name, label in zip(df_data.label_id, df_data['label']):

    fdt_paths_path, category = get_path_and_label(name, label, form = form, prefix = prefix, suffix = suffix)
        

    try:
        img = sitk.ReadImage(fdt_paths_path)
        img_array = sitk.GetArrayFromImage( img)

        if not first_shape_flag:
            first_shape = img_array.shape
            first_shape_flag = True

        elif img_array.shape[0] != first_shape[0] or img_array.shape[1] != first_shape[1] or img_array.shape[2] != first_shape[2]:
            print(img_array.shape)
            continue
    except:
        print(name)
        continue

    labels_data.append(category)
    filenames.append(fdt_paths_path)
    
    mean_img_array = img_array[img_array>0].mean()
    var_img_array = img_array[img_array>0].var()
    
    mean_data.append(mean_img_array)
    var_data.append(var_img_array)
    
assert len(labels_data) == len(filenames)

mean_data = np.mean(mean_data)
std_data = np.sqrt(np.mean(var_data))

batch_size = 8
img_size = 64
test_size = 0.1
val_size = 0.2
val_size = val_size/(1-test_size)


x_train, x_test, y_train,  y_test = train_test_split(filenames, labels_data,  test_size= test_size, random_state=random_state)
x_train, x_val, y_train,  y_val = train_test_split(x_train, y_train,  test_size= val_size, random_state=random_state)

def get_transform(mean, std, mode= 'train'):
    transforms_list = []
    
    def standard(img):
        return (img - mean)/std
    if mode == 'train':
        # transforms_list.append(monai_transforms.RandRotate90(prob = 0.3))
        transforms_list.append(
            monai_transforms.RandGaussianSmooth(
            sigma_x=(0.1, 0.5), 
            sigma_y=(0.1, 0.5), 
            sigma_z=(0.1, 0.5), prob=0.3))
        transforms_list.append(monai_transforms.RandAffine(prob = 0.3, translate_range  =[(-2,2), (-2,2), (-2,2)], padding_mode  = 'zeros'))


    transforms_list.append(torch.Tensor)
    transforms_list.append(standard)
    return monai_transforms.Compose(transforms_list)

train_transform = get_transform(mean_data, std_data, mode = 'train')
val_transform = get_transform(mean_data, std_data, mode = 'val')

train_loader = get_loader(x_train, y_train, train_transform, mode = 'train', batch_size =batch_size, img_size = img_size)
val_loader = get_loader(x_val, y_val,val_transform, mode = 'val', batch_size =batch_size, img_size = img_size)
test_loader = get_loader(x_test, y_test,val_transform, mode = 'test', batch_size =batch_size, img_size = img_size)


normal_0766
normal_0768
normal_0769
mci_0116
mci_0120


# Train

In [13]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = generate_model(10, [64, 128, 256, 512], n_input_channels = 1,conv1_t_size = 5, conv1_t_stride=1, n_classes = 2)
# model = C3D(num_classes = 2)
model = model.to(device)

loss_function = nn.CrossEntropyLoss()


optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
# optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)


In [15]:

writer = SummaryWriter()

current_epoch = 0
epochs = 30
best_f1 = 0
best_auc = 0

savepath = "resnet10_realigned"

for i in range(current_epoch , epochs+1):

    train_loss, preds_prob, labels = epoch_iter(train_loader, model, loss_function, optimizer, device)
    print(f"train_loss: {train_loss};")
    writer.add_scalar('train_loss', train_loss, i)
    _, _ = add_metrics(writer, preds_prob, labels, train_loss, i, mode = 'train')

    val_loss, preds_prob, labels = epoch_iter(val_loader, model, loss_function, optimizer, device, mode = 'val')
    print(f"val_loss: {val_loss};")
    writer.add_scalar('val_loss', val_loss, i)
    f1, rocauc = add_metrics(writer, preds_prob, labels, val_loss, i, mode = 'val')
    if f1>best_f1:
        best_f1 = f1
        save_checkpoint(savepath, model, optimizer, 'f1', i)
    if rocauc>best_auc:
        best_auc = rocauc
        save_checkpoint(savepath, model, optimizer, 'rocauc', i)
    

**************train*************


100%|██████████| 85/85 [00:18<00:00,  4.67it/s]


train_loss: 0.6157857109518612;
loss_train: 0.616;
acc_train: 0.638; bacc_train: 0.638; precision_train: 0.638; recall_train: 0.638; f1_train: 0.638; rocauc_train: 0.707;
**************val*************


100%|██████████| 25/25 [00:02<00:00,  8.79it/s]


val_loss: 0.3561427307128906;
loss_val: 0.356;
acc_val: 0.871; bacc_val: 0.552; precision_val: 0.739; recall_val: 0.552; f1_val: 0.562; rocauc_val: 0.739;
**************train*************


...

100%|██████████| 85/85 [00:17<00:00,  4.80it/s]


train_loss: 0.21440517201143153;
loss_train: 0.214;
acc_train: 0.919; bacc_train: 0.918; precision_train: 0.916; recall_train: 0.918; f1_train: 0.917; rocauc_train: 0.971;
**************val*************


100%|██████████| 25/25 [00:02<00:00,  9.29it/s]


val_loss: 0.6653351593017578;
loss_val: 0.665;
acc_val: 0.737; bacc_val: 0.621; precision_val: 0.574; recall_val: 0.621; f1_val: 0.579; rocauc_val: 0.701;
**************train*************


100%|██████████| 85/85 [00:17<00:00,  6.01it/s]

# Test

In [16]:
savepath = "resnet10_realigned"
checkpoints_dir = f"checkpoints/{savepath}"
checkpoint_type = 'f1'
model_path = os.path.join(checkpoints_dir, f'model-{checkpoint_type}.ckpt')

model_state_dict = torch.load(model_path, map_location=torch.device(device))
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [17]:

mode = 'test'
val_loss, preds_prob, labels = epoch_iter(test_loader, model, loss_function, optimizer, device, mode = 'val')
print(f"val_loss: {val_loss};")
acc, bacc, precision, recall, f1,rocauc = get_metrics(preds_prob, labels)
print(
        f"acc_{mode}: {acc}; bacc_{mode}: {bacc};" +
        f" precision_{mode}: {precision}; recall_{mode}: {recall}; f1_{mode}: {f1};" +
        f" rocauc_{mode}: {rocauc};"
        )

mode = 'val'
val_loss, preds_prob, labels = epoch_iter(val_loader, model, loss_function, optimizer, device, mode = 'val')
print(f"val_loss: {val_loss};")
acc, bacc, precision, recall, f1,rocauc = get_metrics(preds_prob, labels)
print(
        f"acc_{mode}: {acc}; bacc_{mode}: {bacc};" +
        f" precision_{mode}: {precision}; recall_{mode}: {recall}; f1_{mode}: {f1};" +
        f" rocauc_{mode}: {rocauc};"
        )

**************val*************


100%|██████████| 13/13 [00:02<00:00,  6.22it/s]


val_loss: 0.8945473891038161;
acc_test: 0.845; bacc_test: 0.699; precision_test: 0.781; recall_test: 0.699; f1_test: 0.726; rocauc_test: 0.839;
**************val*************


100%|██████████| 25/25 [00:03<00:00,  7.54it/s]

val_loss: 0.692416000366211;
acc_val: 0.84; bacc_val: 0.615; precision_val: 0.638; recall_val: 0.615; f1_val: 0.625; rocauc_val: 0.713;


### Resnet10 top rocauc

val_loss: 0.8527595079862155;
acc_test: 0.619; bacc_test: 0.741; precision_test: 0.661; recall_test: 0.741; f1_test: 0.598; rocauc_test: 0.861;

val_loss: 1.2436859893798828;
acc_val: 0.428; bacc_val: 0.637; precision_val: 0.574; recall_val: 0.637; f1_val: 0.409; rocauc_val: 0.75;

### F1

val_loss: 0.8945473891038161;
acc_test: 0.845; bacc_test: 0.699; precision_test: 0.781; recall_test: 0.699; f1_test: 0.726; rocauc_test: 0.839;

val_loss: 0.692416000366211;
acc_val: 0.84; bacc_val: 0.615; precision_val: 0.638; recall_val: 0.615; f1_val: 0.625; rocauc_val: 0.713;

---

### simple CNN top rocauc

val_loss: 0.7132892608642578;
acc_test: 0.835; bacc_test: 0.693; precision_test: 0.755; recall_test: 0.693; f1_test: 0.715; rocauc_test: 0.824;

val_loss: 0.5662627029418945;
acc_val: 0.835; bacc_val: 0.612; precision_val: 0.629; recall_val: 0.612; f1_val: 0.62; rocauc_val: 0.754;

### F1

val_loss: 1.0085984743558443;
acc_test: 0.825; bacc_test: 0.612; precision_test: 0.774; recall_test: 0.612; f1_test: 0.634; rocauc_test: 0.821;

val_loss: 0.6464328002929688;
acc_val: 0.845; bacc_val: 0.618; precision_val: 0.648; recall_val: 0.618; f1_val: 0.63; rocauc_val: 0.71;

---


In [7]:
### simple CNN top rocauc

val_loss: 0.7132892608642578;
acc_test: 0.835; bacc_test: 0.693; precision_test: 0.755; recall_test: 0.693; f1_test: 0.715; rocauc_test: 0.824;